Download the [ChatGPT knowledge-graph triplet formatted datasets](https://huggingface.co/datasets/mesolitica/chatgpt-kg-triplets) from Huggingface first if you haven't already. Uncomment and run cell below to download the astroawani dataset and save it to the current working directory.

In [25]:
#!wget https://huggingface.co/datasets/mesolitica/chatgpt-kg-triplets/resolve/main/kg-astroawani.translated.jsonl

In [26]:
import json

In [27]:
def rebel_format(triplets):
    """
    Convert
    [['Bruno Santana', 'participant of', '2004 Summer Olympics'],
    ['Bruno Santana', 'participant of', '2008 Summer Olympics'],
    ['Bruno Santana', 'country of citizenship', 'Brazil']]
    to rebel format,
    <triplet> Bruno Santana <subj> 2004 Summer Olympics <obj> participant of <subj> 2008 Summer Olympics <obj> participant of <subj> Brazil <obj> country of citizenship
    """
    q = []
    for no, triple in enumerate(triplets):
        obj = ['<obj>'] + triple[1].split()
        subj = ['<subj>'] + triple[2].split()
        if no > 0 and triple[0] == triplets[no - 1][0]:
            q.extend(subj + obj)
        else:
            triplet = ['<triplet>'] + triple[0].split()
            q.extend(triplet + subj + obj)
    return ' '.join(q)

In [28]:
with open('kg-astroawani.translated.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        break

Let's inspect the data first before we convert it into the rebel format

In [29]:
l.keys()

dict_keys(['id', 'title', 'description', 'body', 'title_kg', 'description_kg', 'body_kg', 'title_kg_ms', 'description_kg_ms', 'body_kg_ms'])

In [30]:
l['title_kg_ms']

[{'head': 'Padah', 'type': 'mempunyai', 'tail': 'hubungan sulit'},
 {'head': 'hubungan sulit', 'type': 'dengan', 'tail': 'pekerja sendiri'},
 {'head': 'Padah', 'type': 'dipecat', 'tail': "CEO McDonald's"}]

In [31]:
data['body_kg_ms'][0]

['Mahkamah Rayuan Singapura pada Selasa menangguhkan keputusannya terhadap rayuan warga Malaysia Nagaentran K. Dharmalingam terhadap keputusan Mahkamah Tinggi.',
 [{'head': 'Mahkamah Rayuan Singapura',
   'type': 'menangguhkan',
   'tail': 'keputusannya'},
  {'head': 'rayuan',
   'type': 'penduduk',
   'tail': 'Malaysia Nagaentran K. Dharmalingam'},
  {'head': 'rayuan', 'type': 'terhadap', 'tail': 'keputusan Mahkamah Tinggi'}]]

In [32]:
with open('processed-kg-astroawani.jsonl', 'w') as fopen_l:
    with open('kg-astroawani.translated.jsonl') as fopen:
        for l in fopen:
            data = json.loads(l)
            
            if data.get('title_kg_ms'):
                triples = []
                for t in data['title_kg_ms']:
                    if any([t_ is None for t_ in t.values()]):
                        continue
                    triples.append([t['head'], t['type'], t['tail']])

                if len(triples):
                    kg = rebel_format(triples)
                    d = {
                        'text': data['title'].strip(),
                        'kg': kg
                    }
                    fopen_l.write(f'{json.dumps(d)}\n')
                    fopen_l.flush()
            
            if data.get('description_kg_ms'):
                triples = []
                for t in data['description_kg_ms']:
                    if any([t_ is None for t_ in t.values()]):
                        continue
                    triples.append([t['head'], t['type'], t['tail']])

                if len(triples):
                    kg = rebel_format(triples)
                    d = {
                        'text': data['description'].strip(),
                        'kg': kg
                    }
                    fopen_l.write(f'{json.dumps(d)}\n')
                    fopen_l.flush()
            
            for body in data['body_kg_ms']:
                if body[1]:
                    triples = []
                    for t in body[1]:
                        if any([t_ is None for t_ in t.values()]):
                            continue
                        triples.append([t['head'], t['type'], t['tail']])
                    
                    if len(triples):
                        kg = rebel_format(triples)
                        d = {
                            'text': body[0].strip(),
                            'kg': kg
                        }
                        fopen_l.write(f'{json.dumps(d)}\n')
                        fopen_l.flush()

In [33]:
# Count the number of lines in the converted dataset
!wc -l processed-kg-astroawani.jsonl

115784 processed-kg-astroawani.jsonl


In [34]:
!head -n 3 processed-kg-astroawani.jsonl

{"text": "Padah jalin hubungan sulit dengan pekerja sendiri, CEO McDonald's dipecat serta merta", "kg": "<triplet> Padah <subj> hubungan sulit <obj> mempunyai <triplet> hubungan sulit <subj> pekerja sendiri <obj> dengan <triplet> Padah <subj> CEO McDonald's <obj> dipecat"}
{"text": "CEO tidak boleh menjalin hubungan dengan mana-mana kakitangan.", "kg": "<triplet> CEO <subj> kakitangan <obj> tidak boleh menjalin hubungan dengan"}
{"text": "SYARIKAT rantaian makanan segera terkemuka dunia, McDonald's Corp mengesahkan telah memecat Ketua Pegawai Eksekutif (CEO), Steve Easterbrook selepas menjalinkan hubungan sulit dengan salah seorang kakitangannya.", "kg": "<triplet> <subj> yang telah memecat Steve Easterbrook <obj> mengesahkan <triplet> Steve Easterbrook <subj> hubungan yang tidak sesuai dengan pekerja <obj> telah"}


In [35]:
!head -n 3 processed-kg-astroawani.jsonl

{"text": "Padah jalin hubungan sulit dengan pekerja sendiri, CEO McDonald's dipecat serta merta", "kg": "<triplet> Padah <subj> hubungan sulit <obj> mempunyai <triplet> hubungan sulit <subj> pekerja sendiri <obj> dengan <triplet> Padah <subj> CEO McDonald's <obj> dipecat"}
{"text": "CEO tidak boleh menjalin hubungan dengan mana-mana kakitangan.", "kg": "<triplet> CEO <subj> kakitangan <obj> tidak boleh menjalin hubungan dengan"}
{"text": "SYARIKAT rantaian makanan segera terkemuka dunia, McDonald's Corp mengesahkan telah memecat Ketua Pegawai Eksekutif (CEO), Steve Easterbrook selepas menjalinkan hubungan sulit dengan salah seorang kakitangannya.", "kg": "<triplet> <subj> yang telah memecat Steve Easterbrook <obj> mengesahkan <triplet> Steve Easterbrook <subj> hubungan yang tidak sesuai dengan pekerja <obj> telah"}
